### Standard KMeans Examples

In [ ]:
!pip install torch-kmeans

In [1]:
# imports
import numpy as np
import torch
from sklearn.datasets import make_blobs
from torch_kmeans import KMeans


In [2]:
# function to generate some clustering data
def get_data(bs: int = 1,
             n: int = 20,
             d: int = 2,
             k: int = 4,
             different_k: bool = False,
             k_lims = (2, 5),
             add_noise: bool = True,
             fp_dtype = torch.float32,
             seed: int = 42):
    torch.manual_seed(seed)
    if different_k:
        a, b = k_lims
        k = torch.randint(low=a, high=b, size=(bs,)).long()
    else:
        k = torch.empty(bs).fill_(k).long()

    # generate pseudo clustering data
    x, y = [], []
    for i, k_ in enumerate(k.numpy()):
        x_, y_ = make_blobs(n_samples=n, centers=k_, n_features=d, random_state=seed+i)
        x.append(x_)
        y.append(y_)
    x = torch.from_numpy(np.stack(x, axis=0))
    y = torch.from_numpy(np.stack(y, axis=0))
    if add_noise:
        x += torch.randn(x.size())

    return x.to(fp_dtype), y, k


In [3]:
# create some data (BS, N, D)
# i.e. 1 instance with N=20 points and D=2 features
BS = 1
K = 4
x, _, _ = get_data(bs=BS, n=20, d=2, k=K, different_k=False)


The inputs to the algorithm must be torch Tensors,
so if your input is e.g. a numpy array, simply use the torch from_numpy() converter
to make it a Tensor:
```python
x_np = np.random.randn(1,2,3)
x_pt = torch.from_numpy(x_np)
```
You can convert back the results again via 'tensor.numpy()'

In [4]:
# initialize model
model = KMeans(n_clusters=K)
print(model)

KMeans(init: 'rnd', num_init: 8, max_iter: 100, distance: LpDistance(), tolerance: 0.0001, normalize: None)


In [5]:
# pytorch style call
result = model(x)
print(result)

Full batch converged at iteration 6/100 with center shifts = tensor([0.]).
ClusterResult(labels=tensor([[2, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 3, 0, 3, 0, 2, 0, 1, 3, 1]]), centers=tensor([[[-9.4776,  7.1169],
         [-7.8595, -6.1823],
         [-2.4830,  8.0605],
         [ 5.6283,  1.2469]]]), inertia=tensor([59.8551]), x_org=tensor([[[ -1.0518,  11.0441],
         [ -3.3334,   6.3465],
         [  4.3055,   1.0529],
         [ -2.3103,   5.4963],
         [ -8.7828,  -4.8557],
         [-10.5590,   6.1168],
         [ -3.7005,   7.9891],
         [ -7.6620,  -7.1754],
         [  5.7378,   1.9245],
         [ -7.9777,  -6.7322],
         [ -9.3876,   6.4422],
         [  6.9063,   3.4280],
         [ -8.0649,   7.8160],
         [  5.3055,  -0.2846],
         [ -7.6417,   7.7096],
         [ -2.0192,   9.4264],
         [-11.7349,   7.4999],
         [ -6.0072,  -5.8342],
         [  5.8864,   0.1140],
         [ -8.8677,  -6.3140]]]), x_norm=tensor([[[ -1.0518,  11.0441],
         [ 

the returned result tuple has many different attributes
associated with the input and the clustering result:

ClusterResult:

"Named and typed result tuple for kmeans algorithms"

- labels: label for each sample in x
- centers: corresponding coordinates of cluster centers
- inertia: sum of squared distances of samples to their closest cluster center
- x_org: original x
- x_norm: normalized x which was used for cluster centers and labels
- k: number of clusters
- soft_assignment: assignment probabilities of soft kmeans


In [6]:
# Instead we can also use scikit-learn style calls:
model = KMeans(n_clusters=K)
fitted_model = model.fit(x)
print(fitted_model.is_fitted)
labels = fitted_model.predict(x)
print(labels)


Full batch converged at iteration 6/100 with center shifts = tensor([0.]).
True
tensor([[2, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 3, 0, 3, 0, 2, 0, 1, 3, 1]])


In [7]:
# or directly
labels = KMeans(n_clusters=K).fit_predict(x)
print(labels)

Full batch converged at iteration 6/100 with center shifts = tensor([0.]).
tensor([[2, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 3, 0, 3, 0, 2, 0, 1, 3, 1]])


In [8]:
# instead of at model initialization, we can also specify the number of clusters
# directly in the forward pass

model = KMeans()
result = model(x, k=K)
print(result.k)


Full batch converged at iteration 6/100 with center shifts = tensor([0.]).
tensor([4])


In [9]:
# the algorithm works for mini-batches of instances
# here we create a batch of 3 instances with N=20 points and D=2 features
BS = 3
K = 4
x, _, _ = get_data(bs=BS, n=20, d=2, k=K, different_k=False)

model = KMeans()
result = model(x, k=K)
print(result.k)
print(result.labels)


Full batch converged at iteration 7/100 with center shifts = tensor([0., 0., 0.]).
tensor([4, 4, 4])
tensor([[2, 2, 3, 2, 1, 0, 2, 1, 3, 1, 0, 3, 0, 3, 0, 2, 0, 1, 3, 1],
        [2, 3, 1, 2, 2, 2, 3, 1, 1, 0, 0, 1, 0, 0, 3, 1, 3, 0, 3, 2],
        [3, 2, 0, 3, 3, 1, 0, 2, 1, 1, 1, 2, 0, 3, 2, 1, 0, 2, 3, 0]])


In [10]:
# we can also provide different numbers of clusters per instance
BS = 3
K = 4
x, _, k_per_instance = get_data(bs=BS, n=20, d=2, k=K, different_k=True)

# in case of different clusters we need to provide a Tensor
# holding the number of clusters per instance
print(k_per_instance)

model = KMeans()
result = model(x, k=k_per_instance)
print(result.k)
print(result.labels)

tensor([2, 4, 3])
Full batch converged at iteration 7/100 with center shifts = tensor([0., 0., 0.]).
tensor([2, 4, 3])
tensor([[0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
        [2, 3, 1, 2, 2, 2, 3, 1, 1, 0, 0, 1, 0, 0, 3, 1, 3, 0, 3, 2],
        [0, 0, 2, 1, 0, 1, 1, 2, 0, 2, 1, 1, 1, 0, 2, 2, 2, 0, 0, 1]])


if we have a good prior idea of where the cluster centers should be located,
e.g. based on domain knowledge, we can provide these centers to the algorithm
to speed up convergence and possibly improve the final quality of the clustering assignment

In [11]:
BS = 1
K = 4
x, y, k_per_instance = get_data(bs=BS, n=20, d=2, k=K, different_k=False)
# here we just use the ground truth labels from the generator
# to set the centers to the group mean + a bit of noise
centers = []
for i in range(K):
    msk = y == i
    centers.append(x[msk].mean(dim=0) + torch.randn(1))
centers = torch.stack(centers).unsqueeze(0) # add batch dimension
print(centers)
# initialize only one run, since we only provide one set of centers
# (otherwise a warning will be raised)
model = KMeans(num_init=1)
result = model(x, k=K, centers=centers)
print(result.centers)


tensor([[[-2.5329,  8.0106],
         [ 6.1546,  1.7733],
         [-7.8680, -6.1908],
         [-8.7486,  7.8460]]])
Full batch converged at iteration 2/100 with center shifts = tensor([0.]).
tensor([[[-2.4830,  8.0605],
         [ 5.6283,  1.2469],
         [-7.8595, -6.1823],
         [-9.4776,  7.1169]]])


One nice feature of torch_kmeans is the possibility to leverage huge
parallelism via execution on GPU.
(On Colab you need to change the runtime type to 'GPU')

To use the GPU if it is available, simply transfer your inputs to the GPU device
and the model will automatically execute the algorithm on GPU.
In order to keep thins simple and easy to maintain, torch_kmeans does not use a dedicated
custom GPU kernel for kmeans but simply leverages the torch tensor operators on GPU
for the most expensive computation steps.
(Most of these steps are JIT compiled via torch.script as well)

In [12]:
assert torch.cuda.is_available()
device = torch.device("cuda")

x_cuda = x.to(device=device)
print(x_cuda.is_cuda, x_cuda.device)

model = KMeans()
result = model(x_cuda, k=K)
# be careful if you use the tensors of the result object
# since now they are located on the GPU:
lbl = result.labels
print(lbl.is_cuda, lbl.device)
# to transfer them back, just use '.cpu()'
lbl = lbl.cpu()
print(lbl.is_cuda, lbl.device)

True cuda:0
Full batch converged at iteration 4/100 with center shifts = tensor([0.], device='cuda:0').
True cuda:0
False cpu


The massive parallelization can for example also be used to very efficiently
find the best number of clusters for a given dataset by computing KMeans for
different numbers of clusters k all in parallel and using the 'Elbow method'
([https://en.wikipedia.org/wiki/Elbow_method_(clustering)](https://en.wikipedia.org/wiki/Elbow_method_(clustering))).


In [13]:
BS = 1
K = 6
x, _, _ = get_data(bs=BS, n=50, d=2, k=6, different_k=False)

assert torch.cuda.is_available()
device = torch.device("cuda")

# replicate instances of x
n_tries = 8
x = x.expand(n_tries, 50, 2)
x_cuda = x.to(device=device)
# use different k between 2 and 10
k_per_isntance = torch.arange(start=2, end=10).to(device=device)

model = KMeans()
result = model(x_cuda, k=k_per_isntance)
# find k according to 'elbow method'
for k, inrt in zip(k_per_isntance, result.inertia.cpu()):
    print(f"k={k}: {inrt}")
# the decrease in inertia after k=6 is much smaller than for the prior steps,
# forming the characteristic 'elbow'

Full batch converged at iteration 8/100 with center shifts = tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0').
k=2: 1549.114990234375
k=3: 426.5956726074219
k=4: 188.72926330566406
k=5: 144.96826171875
k=6: 116.29396057128906
k=7: 101.1533203125
k=8: 85.8719482421875
k=9: 73.476318359375
